In [ ]:
# 第一次运行太长时间，所以修改网络节点个数，降低运行时间
# Adam优化器第一次运行设置0.2的学习率，准去率一直处于0.1左右，第二次修改为1e-4准确率回归正常。

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot = True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

# 初始化权值
def weight_variables(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

# 初始化偏置值
def bias_variables(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x,W):
    # W是一个卷积核 [filter_height,filter_width,in_width,in_channels]
    # strides[0]= strides[3] = 1 规定是1.   strides[1]代表x方向步长 strides[2]代表y轴方向步长
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# 池化层
def max_pool_2x2(x):
    # ksize（窗口大小） = [1,x,y,1] 首位参数默认是1
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

x_image = tf.reshape(x,[-1,28,28,1])

# 5*5 的采样窗口 16个卷积核从1个通道抽取特征
W_conv1 = weight_variables([5,5,1,16])
b_conv1 = bias_variables([16])

h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variables([5,5,16,8])
# 一个卷积核对应一个偏置
b_conv2 = bias_variables([8])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 全连接层  上一层 输出7*7的8个特征图  定义全连接层32个神经元
W_fc1 = weight_variables([7*7*8,32])
b_fc1 = bias_variables([32])

# 把第二个池化层的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*8])
# 第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

keep_pro = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_pro)

W_fc2 = weight_variables([32,10])
b_fc2 = bias_variables([10])

# 计算输出
prediction = tf.matmul(h_fc1_drop,W_fc2) + b_fc2


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
init = tf.global_variables_initializer()
# tf.argmax(vector, 1)：返回的是vector中的最大值的索引号
# 0 按行轴算，取列的最大值，1 按列轴算，取行的最大值
# correct_prediction 是一个布尔型列表
# 使用softmax_cross_entropy_with_logits函数时，用pre作为计算准确率的输入参数
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
# 求准确率
# tf.cast 将correct_prediction 转化为32位的浮点型，True 转为 1 False 转为 0
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_pro:0.7})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_pro:1.0})
        print("Epoch:"+str(epoch)+",Accuracy"+str(acc))

Epoch:0,Accuracy0.794
Epoch:1,Accuracy0.8881
Epoch:2,Accuracy0.9153
Epoch:3,Accuracy0.9264
Epoch:4,Accuracy0.9373
Epoch:5,Accuracy0.9423
Epoch:6,Accuracy0.9466
Epoch:7,Accuracy0.952
Epoch:8,Accuracy0.9564
Epoch:9,Accuracy0.9583
Epoch:10,Accuracy0.9613
Epoch:11,Accuracy0.963
Epoch:12,Accuracy0.9641
Epoch:13,Accuracy0.9663
Epoch:14,Accuracy0.9652
Epoch:15,Accuracy0.968
Epoch:16,Accuracy0.9688
Epoch:17,Accuracy0.9703
Epoch:18,Accuracy0.9706
Epoch:19,Accuracy0.9707
Epoch:20,Accuracy0.9727
